In [1]:
import pandas as pd
import numpy as np
import time, json, requests
from datetime import datetime
import pymysql
from sqlalchemy import create_engine
import FinanceDataReader as fdr
import requests
import sqlalchemy # sql 접근 및 관리를 도와주는 패키지
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request
#pip install -U pykiwoom
from pykiwoom.kiwoom import *
import finterstellar as fs

import warnings
warnings.filterwarnings('ignore')

In [22]:
pip install finterstellar

Note: you may need to restart the kernel to use updated packages.


## Trailing 데이터 생성

In [2]:
def get_trailing(stock_code, period):
    df_quat = {}
    period_quat = period

    for i in range(4):
        print(period_quat, end=" ")
        df_is = get_is(stock_code, period_quat) # 포괄손익계산서
        df_bs = get_bs(stock_code, period_quat) # 재무상태표
        df_cf = get_cf(stock_code, period_quat) # 현금흐름표

        df_merge = pd.merge(df_is, df_bs, how='left', on=['stock_code', 'period', 'rpt_type']) # 
        df_merge = pd.merge(df_merge, df_cf, how='left', on=['stock_code', 'period', 'rpt_type'])

        df_quat[i] = df_merge

        if int(period_quat[5:7])-3 > 0:
            period_quat = period_quat[0:4] + '/' + str(int(period_quat[5:7])-3).zfill(2)

        else:
            period_quat = str(int(period_quat[0:4])-1) + '/12'
        
    df_trailing = pd.DataFrame(df_quat[0], columns=['stock_code', 'period', 'rpt_type'])
    df_trailing[df_quat[0].columns[3:]] = 0

    for i in range(len(df_quat)):
        df_trailing[df_quat[0].columns[3:]] += df_quat[i][df_quat[i].columns[3:]]
        
    return df_trailing

## 코스피 데이터 DB에 저장

In [3]:
def get_kospi_to_db():
    today = datetime.now().date() #오늘 날짜 가져오기
    
    #kopsi 테이블이 없다면 새로 생성
    df_kospi = fdr.StockListing('KOSPI')
    
    sql='CREATE TABLE if NOT EXISTS kospi(stock_code VARCHAR(20),stock_name VARCHAR(500))'
    
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                            user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()
    
    cursor.execute(sql)
    
    server = '127.0.0.1' # local server
    user = 'root' # user name
    password = '1541' # 개인 password
    db = 'termproject' # DB 이름

    # sqlalchemy의 create_engine을 이용해 DB 연결
    engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))
    
    #종목코드와 이름만 가져오기
    is_domestic = pd.DataFrame({'stock_code' : df_kospi['Symbol'] , 'stock_name' : df_kospi['Name']})
    
    is_domestic.to_sql(name='kospi',con=engine,if_exists='append',index=False,
    dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.
                 
                 '종목 코드' : sqlalchemy.types.VARCHAR(20),
                 '종목 이름' : sqlalchemy.types.VARCHAR(1000)
             })
    
    # DB 접속 해제
    engine.dispose()
    cursor.close()
    conn.close()

## 코스닥 데이터 DB에 저장

In [4]:
def get_kosdaq_to_db():
    today = datetime.now().date() #오늘 날짜 가져오기
    
    #kopsi 테이블이 없다면 새로 생성
    df_kosdaq = fdr.StockListing('KOSDAQ')
    
    sql='CREATE TABLE if NOT EXISTS kosdaq(stock_code VARCHAR(20),stock_name VARCHAR(500))'
    
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                            user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()
    
    cursor.execute(sql)
    
    server = '127.0.0.1' # local server
    user = 'root' # user name
    password = '1541' # 개인 password
    db = 'termproject' # DB 이름

    # sqlalchemy의 create_engine을 이용해 DB 연결
    engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))
    
    #종목코드와 이름만 가져오기
    is_domestic = pd.DataFrame({'stock_code' : df_kosdaq['Symbol'] , 'stock_name' : df_kosdaq['Name']})
    
    is_domestic.to_sql(name='kosdaq',con=engine,if_exists='append',index=False,
    dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.
                 
                 '종목 코드' : sqlalchemy.types.VARCHAR(20),
                 '종목 이름' : sqlalchemy.types.VARCHAR(1000)
             })
    
    # DB 접속 해제
    engine.dispose()
    cursor.close()
    conn.close()

## 수정주가 데이터 저장

In [5]:
def get_adjuseted_to_db(start_date,end_date,stock_code):
    
    #fdr.DataReader를 이용해 수정주가 데이터 가져오기
    df_stock = fdr.DataReader(stock_code,start=start_date,end=end_date)
    
    server = '127.0.0.1' # local server
    user = 'root' # user name
    password = '1541' # 개인 password
    db = 'termproject' # DB 이름

    # sqlalchemy의 create_engine을 이용해 DB 연결
    engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))
    
    #테이블 이름은 종목코드_adgected
    table_name=stock_code+'_Adjected'
    
    #df_stock 의 index를 추가
    df_stock['Date'] = df_stock.index
    
    #columns 재배치
    df_stock=df_stock[['Date','Open', 'High', 'Low', 'Close', 'Volume', 'Change']]
    
    df_stock.to_sql(name=table_name,con=engine,if_exists='append',index=False,
                   dtype={
                         'Date' : sqlalchemy.types.DATE(),
                         'Open' : sqlalchemy.types.BIGINT(),
                         'High' : sqlalchemy.types.BIGINT(),
                         'Low' : sqlalchemy.types.BIGINT(),
                         'Close' : sqlalchemy.types.BIGINT(),
                         'Volume' : sqlalchemy.types.BIGINT(),
                         'Change' : sqlalchemy.types.FLOAT(),
                   })
    
    # DB 접속 해제
    engine.dispose()

## 손익계산서 저장

In [6]:
def get_income_statement_to_db(stock_code):
    
    #연결 손익계산서 : D=&MenuYn=Y
    #별도 손익계산서 : B=&MenuYn=Y
    url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A"+stock_code+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701"
    
    headers ={"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36 Edg/100.0.1185.29"}

    #크롤링 시작
    req = Request(url = url, headers = headers)

    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')

    freq = 'A'
    
    if freq.upper() == 'A':
        is_a = soup.find(id ='divSonikY')
        num_col = 4
    else :
        is_a = soup.find(id=  'divSonikQ')
        num_col = 4
    
    #tr태그인것 모으기
    is_a = is_a.find_all(['tr'])
    
    items_kr =[is_a[m].find(['th']).get_text().replace('\n','').replace('\xa0','').replace('계산에 참여한 계정 펼치기','')
               for m in range(1,len(is_a))]
    
    #기간 추출
    #4개 분기 보고서를 제공받음
    period = [is_a[0].find_all('th')[n].get_text() for n in range(1, num_col+1)]
    
    items_en =['rev','cgs','gross','sga','sga1','sga2','sga3','sga4','sga5','sga6','sga7','sga8','opr','opr_',
           'fininc','fininc1','fininc2','fininc3','fininc4','fininc5','fininc6','fininc7','fininc8','fininc9','fininc10','fininc11',
          'fincost','fincost1','fincost2','fincost3','fincost4','fincost5','fincost6','fincost7','fincost8','fincost9','fincost10',
          'otherrev','otherrev1','otherrev2','otherrev3','otherrev4','otherrev5','otherrev6','otherrev7','otherrev8','otherrev9','otherrev10','otherrev11','otherrev12','otherrev13','otherrev14','otherrev15','otherrev16',
          'othercost','othercost1','othercost2','othercost3','othercost4','othercost5','othercost6','othercost7','othercost8','othercost9','othercost10','othercost11','othercost12','otherpl','otherpl1','otherpl2','otherpl3','otherpl4',
          'ebit','tax','contop','discontop','netinc']
    items_en = items_en + ['netinc1','netinc2']
    
    for item, i in zip(items_en, range(1, len(is_a))):
        temps = []
        for j in range(0,num_col):
            temp = [float(is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))
                   if is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''
                    else (0 if is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0'
                    else 0)]
            temps.append(temp[0])

        globals()[item] = temps
    
    is_domestic = pd.DataFrame({'종목':stock_code,'기간':period,'매출액':rev, '매출원가':cgs, '매출총이익':gross,'판매비와관리비':sga,
                           '영업이익':opr, '영업이익(발표기준)': opr_,'금융수익' : fininc, '금융원가' : fincost, '기타수익' : otherrev, 
                            '기타비용' : othercost, '관계기업관련손익' : otherpl, 'EBIT' : ebit, '법인세비용' : tax, '계속영업이익' : contop,
                            '중단영업이익':discontop, '당기순이익' : netinc, '지배주주순이익' : globals()['netinc1'], '비지배주주순이익' : globals()['netinc2']
                           })
    # 크롤링 완료
    
    server = '127.0.0.1' # local server
    user = 'root' # user name
    password = '1541' # 개인 password
    db = 'termproject' # DB 이름

    # sqlalchemy의 create_engine을 이용해 DB 연결
    engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))
    
    #테이블 이름은 종목코드_adgected
    table_name=stock_code+'_Income_Statement'
    
    #DB저장
    is_domestic.to_sql(name=table_name,con=engine,if_exists='append',index=False,
                   dtype={
                        '종목' : sqlalchemy.types.VARCHAR(6),
                        '기간' : sqlalchemy.types.VARCHAR(8),
                        '매출액' : sqlalchemy.types.FLOAT(),
                        '매출원가' : sqlalchemy.types.FLOAT(),
                        '매출총이익' : sqlalchemy.types.FLOAT(),
                        '판매비와관리비' : sqlalchemy.types.FLOAT(),
                        '영업이익' : sqlalchemy.types.FLOAT(),
                        '영업이익(발표기준)' : sqlalchemy.types.FLOAT(),
                        '금융수익' : sqlalchemy.types.FLOAT(),
                        '금융원가' : sqlalchemy.types.FLOAT(),
                        '기타수익' : sqlalchemy.types.FLOAT(),
                        '기타비용' : sqlalchemy.types.FLOAT(),
                        '관계기업관련손익' : sqlalchemy.types.FLOAT(),
                        'EBIT' : sqlalchemy.types.FLOAT(),
                        '법인세비용' : sqlalchemy.types.FLOAT(),
                        '계속영업이익' : sqlalchemy.types.FLOAT(),
                        '중단영업이익' : sqlalchemy.types.FLOAT(),
                        '당기순이익' : sqlalchemy.types.FLOAT(),
                        '지배주주순이익' : sqlalchemy.types.FLOAT(),
                        '비지배주주순이익' : sqlalchemy.types.FLOAT()
                   })
    
    # DB 접속 해제
    engine.dispose()

## 재무상태표 저장

In [7]:
def get_statement_of_position_to_db(stock_code, rpt_type, freq):
    
    # stock_code : 주식 코드
    # rpt_trpe : 연결(Consolidated) / 별도(Unconsolidated) 구분
    # freq = 연간(A) / 분기(Q)
    
    items_en = [ 'assets','curassets','curassets1','curassets2','curassets3','curassets4','curassets5', #7
                'curassets6','curassets7','curassets8','curassets9','curassets10','curassets11', #6
                'ltassets','ltassets1','ltassets2','ltassets3','ltassets4','ltassets5','ltassets6', 'ltassets7', #8
                'ltassets8','ltassets9','ltassets10','ltassets11','ltassets12','ltassets13','finassets', #7
                'liab','curliab','curliab1','curliab2','curliab3','curliab4','curliab5', #7
                'curliab6','curliab7','curliab8','curliab9','curliab10','curliab11','curliab12','curliab13',#8
                'ltliab','ltliab1','ltliab2','ltliab3','ltliab4','ltliab5','ltliab6', #7
                'ltliab7','ltliab8','ltliab9','ltliab10','ltliab11','ltliab12','finliab', #7
                'equity','equity1','equity2','equity3','equity4','equity5','equity6','equity7','equity8'] #9
    
    if rpt_type.upper() == 'CONSOLIDATED' :
        url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode=A'+stock_code+'&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701'
        
    else : 
        url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A'+stock_code+'&cID=&MenuYn=Y&ReportGB=B&NewMenuID=103&stkGb=701'
        items_en=[item for item in items_en if item not in ['equity1','equity8']]

    headers ={"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36 Edg/100.0.1185.29"}

    #크롤링 시작
    req = Request(url = url, headers = headers)

    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    if freq.upper() =='A': #연간 재무상태표 영역 추출
        bs_a = soup.find(id = 'divDaechaY')
        num_col = 3 #최근 3개 연간 데이터
        
    else :
        bs_a = soup.find(id='divDaechaQ')
        num_col = 4 #최근 4개 분기 데이터
        
    bs_a = bs_a.find_all(['tr'])
    
    #연간 재무상태표 항목 쳘핀 뒤 하위 항목 추출
    items_kr = [bs_a[m].find(['th']).get_text().replace('\n','').replace('\xa0','').replace('계산에 참여한 계정 펼치기','')
               for m in range(1,len(bs_a))]
    
    #최근 3개 연간 재무상태표 자료 수집(첫 번째부터 세 번째 컬럼까지 수집)
    period = [bs_a[0].find_all('th')[n].get_text() for n in range(1, num_col+1)]
    
    for item , i in zip(items_en , range(1,len(bs_a))):
        #항목별 최근 3개 연간데이터 불러와서 temps에 모으기
        
        temps=[]
        for j in range(0, num_col): # 첫 번째 부터 세 번째 컬럼까지 수집
            temp = [float(bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))
                   if bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''
                   else (0 if bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0'
                        else 0)]
            temps.append(temp[0])
            
        #item_en 내 각 항목을 global 변수로 지정하고 temps값 저장
        globals()[item] = temps
        
    #지배 / 비지배 항목 처리
    if rpt_type.upper() == 'CONSOLIDATED': #연결 연간 재무상태표는 아무 것도 하지 않음
        pass
    else : #별도 연간 재무상태표는 연간에만 존재하는 항목을 Null값으로 채움
        globals()['equity1'],globals()['equity8'] = [np.NaN]*num_col , [np.NaN]*num_col
        
    #재무상태표 각 컬럼에 위에서 저장한 global 변수값을 지정하여 dataframe으로 변환
    bs_domestic = pd.DataFrame({
        '자산':assets,
        '유동자산':curassets,
        '비유동자산': ltassets,
        '기타금융업자산': finassets,
        '부채':liab,
        '유동부채':curliab,
        '비유동부채':ltliab,
        '기타금융업부채':finliab,
        '자본':equity,
        '자본금': equity2, 
        '신종자본증권': equity3,
        '자본잉여금':equity4,
        '기타자본':equity5,
        '기타포괄손익누계액':equity6,
        '이익잉여금(결손금)':equity7,
        '기간' : period
    })
        
    bs_domestic['rpt_type'] = rpt_type+'_'+freq.upper()
    
    #DB 시작
    server = '127.0.0.1' # local server
    user = 'root' # user name
    password = '1541' # 개인 password
    db = 'termproject' # DB 이름

    # sqlalchemy의 create_engine을 이용해 DB 연결
    engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))
    
    #테이블 이름은 종목코드_adgected
    table_name=stock_code+'_statement_of_position'
    
    bs_domestic.to_sql(name=table_name,con=engine,if_exists='append',index=False,
                   dtype={
                        '자산' : sqlalchemy.types.FLOAT(),
                        '유동자산' : sqlalchemy.types.FLOAT(),
                        '비유동자산' : sqlalchemy.types.FLOAT(),
                        '기타금융업자산' : sqlalchemy.types.FLOAT(),
                        '부채' : sqlalchemy.types.FLOAT(),
                        '유동부채' : sqlalchemy.types.FLOAT(),
                        '비유동부채' : sqlalchemy.types.FLOAT(),
                        '기타금융업부채' : sqlalchemy.types.FLOAT(),
                        '자본' : sqlalchemy.types.FLOAT(),
                        '자본금' : sqlalchemy.types.FLOAT(),
                        '신종자본증권' : sqlalchemy.types.FLOAT(),
                        '자본잉여금' : sqlalchemy.types.FLOAT(),
                        '기타자본' : sqlalchemy.types.FLOAT(),
                        '기타포괄손익누계액' : sqlalchemy.types.FLOAT(),
                        '이익잉여금(결손금)' : sqlalchemy.types.FLOAT(),
                        'rpt_type' : sqlalchemy.types.VARCHAR(15),
                        '기간' : sqlalchemy.types.VARCHAR(8)
                   })
    
    # DB 접속 해제
    engine.dispose()
    
    
        

## 현금흐름표 저장

In [8]:
def get_cashflow_statement_to_db(stock_code, rpt_type, freq):
    
    # stock_code : 주식 코드
    # rpt_trpe : 연결(Consolidated) / 별도(Unconsolidated) 구분
    # freq = 연간(A) / 분기(Q)
    
    items_en = ["cfo","cfo1","cfo2","cfo3","cfo4","cfo5","cfo6","cfo7", # 8개
                "cfi","cfi1","cfi2","cfi3","cff","cff1","cff2","cff3",#8개
                "cff4","cff5","cff6","cff7","cff8","cff9" #6개
               ]
    
    if rpt_type.upper() == 'CONSOLIDATED' :
        url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode=A'+stock_code+'&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701'
        
    else : 
        url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A'+stock_code+'&cID=&MenuYn=Y&ReportGB=B&NewMenuID=103&stkGb=701'
    
    headers ={"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36 Edg/100.0.1185.29"}

    #크롤링 시작
    req = Request(url = url, headers = headers)

    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    if freq.upper() == 'A': #연간 현금흐름표 영역 추출
        cf_a = soup.find(id ='divCashY')
        num_col = 3 #최근 3개 연간 데이터
    else : #분기 현금흐름표 영역 추출(freq.upper() == 'Q')
        cf_a = soup.find(id='divCashQ')
        num_col = 4 #최근 4개 분기 데이터
    cf_a = cf_a.find_all(['tr'])
    
    #연간 현금흐름표 항목 펼친 뒤 하위 항목 추출
    items_kr = [cf_a[m].find(['th']).get_text().replace('\n','').replace('\xa0','').replace('계산에 참여한 계정 펼치기','')
               for m in range(1,len(cf_a))]
    
    #최근 3개 연간 현금흐름표 자료 수집( 첫 번째부터 세 번째 컬럼까지 수집)
    period = [cf_a[0].find_all('th')[n].get_text() for n in range(1,num_col+1)]
    
    #항목별 값 불러오기)수집할 인덱스 값 미리 설정)
    
    idx=[1,2,3,4,39,70,75,76,84,85,99,113,121,122,134,145,153,154,155,156,157,158]

    for item, i in zip(items_en, idx) :
        
        temps=[]
        for j in range(0, num_col):
            temp = [float(cf_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))
                   if cf_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''
                   else(0 if cf_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0'
                       else 0)]
            temps.append(temp[0])

            #item_en 내 각 항목을 global 변수로 지정하고 값 저장
            globals()[item]=temps
            
    cf_domestic = pd.DataFrame({"기간":period,
                                "영업활동으로인한현금흐름":cfo,
                                "당기순손익":cfo1,
                                "법인세비용차감전계속사업이익":cfo2,
                                "현금유출이없는비용등가산":cfo3,
                                "(현금유입이없는수익등차감)":cfo4,
                                "영업활동으로인한자산부채변동(운전자본변동)":cfo5,
                                "*영업에서창출된현금흐름" : cfo6,
                                "기타영업활동으로인한현금흐름" : cfo7,
                                "투자활동으로인한현금흐름":cfi,
                                "투자활동으로인한현금유입액" : cfi1,
                                "(투자활동으로인한현금유출액)" : cfi2,
                                "기타투자활동으로인한현금흐름" : cfi3,
                                "재무활동으로인한현금흐름":cff,
                                "재무활동으로인한현금유입액" : cff1,
                                "(재무활동으로인한현금유출액)" : cff2,
                                "기타재무활동으로인한현금흐름" : cff3,
                                "영업투자재무활동기타현금흐름" : cff4,
                                "연결범위변동으로인한현금의증가" : cff5,
                                "환율변동효과" : cff6,
                                "현금및현금성자산의증가" : cff7,
                                "기초현금및현금성자산" : cff8,
                                "기말현금및현금성자산" : cff9
                               })
    
    cf_domestic['rpt_type'] = rpt_type
    
    
    #DB 시작
    server = '127.0.0.1' # local server
    user = 'root' # user name
    password = '1541' # 개인 password
    db = 'termproject' # DB 이름

    # sqlalchemy의 create_engine을 이용해 DB 연결
    engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))
    
    #테이블 이름은 종목코드_adgected
    table_name=stock_code+'_Cash_Flow_Statement'
    
    cf_domestic.to_sql(name=table_name,con=engine,if_exists='append',index=False,
                   dtype={
                        '기간' : sqlalchemy.types.VARCHAR(8),
                        '영업활동으로인한현금흐름' : sqlalchemy.types.FLOAT(),
                        '당기순손익' : sqlalchemy.types.FLOAT(),
                        '법인세비용차감전계속사업이익' : sqlalchemy.types.FLOAT(),
                        '현금유출이없는비용등가산' : sqlalchemy.types.FLOAT(),
                        '(현금유입이없는수익등차감)' : sqlalchemy.types.FLOAT(),
                        '영업활동으로인한자산부채변동(운전자본변동)' : sqlalchemy.types.FLOAT(),
                        '*영업에서창출된현금흐름' : sqlalchemy.types.FLOAT(),
                        '기타영업활동으로인한현금흐름' : sqlalchemy.types.FLOAT(),
                        '투자활동으로인한현금흐름' : sqlalchemy.types.FLOAT(),
                        '투자활동으로인한현금유입액' : sqlalchemy.types.FLOAT(),
                        '(투자활동으로인한현금유출액)' : sqlalchemy.types.FLOAT(),
                        '기타투자활동으로인한현금흐름' : sqlalchemy.types.FLOAT(),
                        '재무활동으로인한현금흐름' : sqlalchemy.types.FLOAT(),
                        '재무활동으로인한현금유입액' : sqlalchemy.types.FLOAT(),
                        '(재무활동으로인한현금유출액)' : sqlalchemy.types.FLOAT(),
                        '기타재무활동으로인한현금흐름' : sqlalchemy.types.FLOAT(),
                        '영업투자재무활동기타현금흐름' : sqlalchemy.types.FLOAT(),
                        '연결범위변동으로인한현금의증가' : sqlalchemy.types.FLOAT(),
                        '환율변동효과' : sqlalchemy.types.FLOAT(),
                        '현금및현금성자산의증가' : sqlalchemy.types.FLOAT(),
                        '기초현금및현금성자산' : sqlalchemy.types.FLOAT(),
                        '기말현금및현금성자산' : sqlalchemy.types.VARCHAR(15)
                   })
    
    # DB 접속 해제
    engine.dispose()

## 데이터베이스에 자동으로 넣는 코드

In [9]:
def get_all_kosdaq_to_db():
    sql='SELECT stock_code FROM kosdaq'

    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    #최근 3년간 데이터 수집
    start_date = '2019-06-01'
    end_date = '2022-06-01'

    for str in cursor :
        for stock_code in str :
            try:
                get_adjuseted_to_db(start_date,end_date,stock_code)
                get_incom_statement_to_db(stock_code)
                get_statement_of_position_to_db(stock_code,'CONSOLIDATED','A')
                get_cashflow_statement_to_db(stock_code,'CONSOLIDATED','A')
            except:
                pass

    #DB접속 해제
    cursor.close()
    conn.close()

In [10]:
def get_all_kospi_to_db():

    sql='SELECT stock_code FROM kospi'

    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    #최근 3년간 데이터 수집
    start_date = '2019-06-01'
    end_date = '2022-06-01'

    for str in cursor :
        for stock_code in str :
            try:
                get_adjuseted_to_db(start_date,end_date,stock_code)
                get_incom_statement_to_db(stock_code)
                get_statement_of_position_to_db(stock_code,'CONSOLIDATED','A')
                get_cashflow_statement_to_db(stock_code,'CONSOLIDATED','A')
            except:
                pass

    #DB접속 해제
    cursor.close()
    conn.close()

## 시가총액/ 상장주식 수 가져오기

In [11]:
def get_stock_data_to_db(date) :

    url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
    headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.108 Whale/3.15.136.18 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest'
    }

    p_data = {
        'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501',
        'locale': 'ko_KR',
        'mktId': 'ALL',
        'trdDd': date,
        'share': '1',
        'money': '1',
        'csvxls_isNo': 'false',
    }



    res = requests.post(url,headers=headers, data = p_data)

    html_text = res.content

    html_jsons = json.loads(html_text)

    html_jsons = html_jsons['OutBlock_1']

    daily = []

    if len(html_jsons) > 0 :
        for html_json in html_jsons: 
            if html_json['TDD_OPNPRC'] == '-': #시장이 열리지 않아 값이 없는 경우
                continue

            ISU_SRT_CD = html_json['ISU_SRT_CD'] # 종목 코드
            ISU_ABBRV = html_json['ISU_ABBRV'] #종목 이름
            TRD_DD = date # 날짜

            MKTCAP = float(html_json['MKTCAP'].replace(',','')) #시가총액
            LIST_SHRS = float(html_json['LIST_SHRS'].replace(',','')) #상장주식 수

            daily.append((ISU_SRT_CD,ISU_ABBRV,TRD_DD,MKTCAP,LIST_SHRS))
    else :
        pass

    if len(daily) > 0:
        daily = pd.DataFrame(daily)
        daily.columns = ['stock_code','stock_name','Date','시가총액','상장주식수']

    #DB 시작
    server = '127.0.0.1' # local server
    user = 'root' # user name
    password = '1541' # 개인 password
    db = 'termproject' # DB 이름

    # sqlalchemy의 create_engine을 이용해 DB 연결
    engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))

    #테이블 이름은 종목코드_adgected
    table_name='stock_data_'+date

    try:
        daily.to_sql(name=table_name,con=engine,if_exists='append',index=False,
                       dtype={
                            'stock_code' : sqlalchemy.types.VARCHAR(8),
                            'stock_name' : sqlalchemy.types.VARCHAR(30),
                            'Date' : sqlalchemy.types.Date(),
                            '시가총액' : sqlalchemy.types.FLOAT(),
                            '상장주식수' : sqlalchemy.types.FLOAT(),

                       })
        
    except: #예외처리
        print("수집하려는 그 날은 주식시장이 쉬는날입니다!")

    # DB 접속 해제
    engine.dispose()


## PER

In [12]:
def get_PER(year):
    
    #시가총액 순으로 100개의 기업만 가져온다.
    
    if year == '2019':
        sql = "SELECT stock_code FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
    elif year == '2020':
        sql = "SELECT stock_code FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
    elif year == '2021':
        sql = "SELECT stock_code FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
    else : #2022
        sql = "SELECT stock_code FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"

    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    stock_codes=[]

    #DB에서 code만 가져오기
    for str in cursor :
        for stock_code in str:
            stock_codes.append(stock_code)

    df_factor = pd.DataFrame()

    #기간 , 당기순이익 가져오기
    for stock_code in stock_codes:
        try:
            #2022만 3월 나머지는 12월
            if year == '2022':
                sql = 'SELECT 기간, 당기순이익 FROM '+ stock_code+"_income_statement WHERE 기간 = '"+year+"/03'"

            else:
                sql = 'SELECT 기간, 당기순이익 FROM '+ stock_code+"_income_statement WHERE 기간 = '"+year+"/12'"
            
            cursor.execute(sql)

            for period,profit in cursor:
                df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period,
                    '당기순이익' : float(profit)
                } , ignore_index=True)


        except :
            pass

            
    #EPS 계산 후 추가
    for index, row in df_factor.iterrows():
        if row['기간'][:4] == year:
            if year == '2019':
                sql = 'SELECT 상장주식수 FROM stock_data_20191202 WHERE stock_code = ' +row['stock_code']+ ';'
            elif year == '2020':
                sql = 'SELECT 상장주식수 FROM stock_data_20201201 WHERE stock_code = ' +row['stock_code']+ ';'
            elif year == '2021':
                sql = 'SELECT 상장주식수 FROM stock_data_20211201 WHERE stock_code = ' +row['stock_code']+ ';'
            elif year == '2022':
                sql = 'SELECT 상장주식수 FROM stock_data_20220302 WHERE stock_code = ' +row['stock_code']+ ';'

        cursor.execute(sql)

        
        #0으로 나누는 경우 제외
        try:
            #당기순이익을 상장주식수로 나누어준다
            for num in cursor:
                EPS = row['당기순이익'] * 100000000 / num[0]
                df_factor.loc[index,'EPS'] = EPS
        except:
            pass

        
    #PER 계산 후 추가
    for index, row in df_factor.iterrows():
        if row['기간'][:4] == year :
            if year == '2019':
                sql = "SELECT Close FROM "+row['stock_code']+"_adjected WHERE DATE = '2019-12-02';"
            elif year == '2020':
                sql = "SELECT Close FROM "+row['stock_code']+"_adjected WHERE DATE = '2020-12-01';"
            elif year == '2021':
                sql = "SELECT Close FROM "+row['stock_code']+"_adjected WHERE DATE = '2021-12-01';"
            elif year == '2022':
                sql = "SELECT Close FROM "+row['stock_code']+"_adjected WHERE DATE = '2022-03-02';"

        cursor.execute(sql)
        
        #0으로 나누는 경우 제외
        try:
            for close in cursor:
                PER = close[0] /row['EPS']
                df_factor.loc[index,'PER'] = PER
        except:
            pass
                

    df_factor = df_factor.sort_values(by=['PER'])

    top_50 = df_factor[:50]    

    #DB접속 해제
    cursor.close()
    conn.close()
    
    return top_50

## PBR

In [13]:
def get_PBR(year):
    
    #시가총액 순으로 100개의 기업만 가져온다.
    
    if year == '2022':
        print("2022의 재무상태표가 아직 없으므로 계산할 수 없습니다!")
        return
    
    if year == '2019':
        sql = "SELECT stock_code FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
    elif year == '2020':
        sql = "SELECT stock_code FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
    elif year == '2021':
        sql = "SELECT stock_code FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
    else : #2022
        sql = "SELECT stock_code FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"

    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    stock_codes=[]
    #DB에서 code만 가져오기
    for str in cursor :
        for stock_code in str:
            stock_codes.append(stock_code)

    df_factor = pd.DataFrame()
    
    #기간 , 당기순이익 가져오기
    for stock_code in stock_codes:
        try:
            #2022만 3월 나머지는 12월
            if year == '2022':
                sql = 'SELECT 기간, 자본 FROM '+ stock_code+"_statement_of_position WHERE 기간 = '"+year+"/03'"

            else:
                sql = 'SELECT 기간, 자본 FROM '+ stock_code+"_statement_of_position WHERE 기간 = '"+year+"/12'"

            cursor.execute(sql)

            for period,profit in cursor:
                df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period,
                    '자본' : float(profit)
                } , ignore_index=True)


        except :
            pass

    
    
        #BPS 계산 후 추가
        for index, row in df_factor.iterrows():
            if row['기간'][:4] == year:
                if year == '2019':
                    sql = 'SELECT 상장주식수 FROM stock_data_20191202 WHERE stock_code = ' +row['stock_code']+ ';'
                elif year == '2020':
                    sql = 'SELECT 상장주식수 FROM stock_data_20201201 WHERE stock_code = ' +row['stock_code']+ ';'
                elif year == '2021':
                    sql = 'SELECT 상장주식수 FROM stock_data_20211201 WHERE stock_code = ' +row['stock_code']+ ';'
                elif year == '2022':
                    sql = 'SELECT 상장주식수 FROM stock_data_20220302 WHERE stock_code = ' +row['stock_code']+ ';'

            cursor.execute(sql)

            #0으로 나누는 경우 제외
            try:
                for num in cursor:
                    BPS = row['자본'] * 100000000 / num[0]
                    df_factor.loc[index,'BPS'] = BPS

            except:
                pass

    #PBR 계산 후 추가
    for index, row in df_factor.iterrows():
        if row['기간'][:4] == year :
            if year == '2019':
                sql = "SELECT Close FROM "+row['stock_code']+"_adjected WHERE DATE = '2019-12-02';"
            elif year == '2020':
                sql = "SELECT Close FROM "+row['stock_code']+"_adjected WHERE DATE = '2020-12-01';"
            elif year == '2021':
                sql = "SELECT Close FROM "+row['stock_code']+"_adjected WHERE DATE = '2021-12-01';"
            elif year == '2022':
                sql = "SELECT Close FROM "+row['stock_code']+"_adjected WHERE DATE = '2022-03-02';"

        cursor.execute(sql)

        #0으로 나누는 경우 제외
        try:
            for close in cursor:
                PBR = close[0] /row['BPS']
                df_factor.loc[index,'PBR'] = PBR

        except:
            pass
                
                
    df_factor = df_factor.sort_values(by=['PBR'])

    top_50 = df_factor[:50]        

    #DB접속 해제
    cursor.close()
    conn.close()        
    
    return top_50

## PSR

In [14]:
def get_PSR(year):
    
    #시가총액 순으로 100개의 기업만 가져온다.
    #PSR = 주가매출액비율
    #PSR = 주가/주당매출액 = (주가*주식수) / (주당매출액 *주식수) = 시가총액 / 매출액
    
    if year == '2019':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2019/12'
    elif year == '2020':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2020/12'
    elif year == '2021':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2021/12'
    else : #2022
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2022/03'
        
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    df_factor = pd.DataFrame()
    
    for stock_code, total in cursor :
        df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period,
                    '시가총액' : float(total)
                } , ignore_index=True)
        
        
    for index, row in df_factor.iterrows():
        sql='SELECT 매출액 FROM '+row['stock_code']+"_income_statement WHERE 기간 = '"+period+"';"
    
        #수집안된 데이터 있는경우 패스
        try:
            cursor.execute(sql)        
            for sells in cursor:
                PSR = row['시가총액'] /(sells[0]*100000000)
                df_factor.loc[index,'PSR'] = PSR
            
        except:
            pass
        

    #PSR이 낮을수록 저평가되었다고 판단
    df_factor = df_factor.sort_values(by=['PSR'])

    top_50 = df_factor[:50]        
    
    
    #DB접속 해제
    cursor.close()
    conn.close()        
    
    return top_50

## PCR

In [15]:
def get_PCR(year):
    
    #시가총액 순으로 100개의 기업만 가져온다.
    #PCR = 주가현금흐름비율
    #PCR = 주가/주당영업현금흐름 = (주가*주식수) / (주당영업현금흐름 *주식수) = 시가총액 / 영업현금흐름
    
    if year == '2019':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2019/12'
    elif year == '2020':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2020/12'
    elif year == '2021':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2021/12'
    else : #2022
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2022/03'
        
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    df_factor = pd.DataFrame()
    
    for stock_code, total in cursor :
        df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period,
                    '시가총액' : float(total)
                } , ignore_index=True)
    
    
    
    for index, row in df_factor.iterrows():
        sql='SELECT 영업활동으로인한현금흐름 FROM '+row['stock_code']+"_cash_flow_statement WHERE 기간 = '"+period+"';"
        
        #수집안된 데이터 있는경우 패스
        try:
            cursor.execute(sql)        
            for cashflow in cursor:
                PCR = row['시가총액'] /(cashflow[0]*100000000)
                df_factor.loc[index,'PCR'] = PCR
        
        except:
            pass
    
    #PCR이 낮을수록 저평가되었다고 판단
    df_factor = df_factor.sort_values(by=['PCR'])

    top_50 = df_factor[:50]        
    
    #DB접속 해제
    cursor.close()
    conn.close()        
    
    return top_50

## 가치지표 결합

In [16]:
def mix_index(index1,index2,year) :
    index1 = index1.upper()
    index2 = index2.upper()
    
    #첫 번째 df
    if index1 == 'PER' :
        df1 = get_PER(year)
        
    elif index1 == 'PBR' :
        df1 = get_PBR(year)
        
    elif index1 == 'PCR' :
        df1 = get_PCR(year)
        
    else : # PSR
        df1 = get_PSR(year)
        
    #두 번째 df
    if index2 == 'PER' :
        df2 = get_PER(year)
        
    elif index2 == 'PBR' :
        df2 = get_PBR(year)
        
    elif index2 == 'PCR' :
        df2 = get_PCR(year)
        
    else : # PSR
        df2 = get_PSR(year)
      
      
    #서로의 df내의 교집합 찾기
    combi = fs.combine_signal(df1,df2, how='and')
    
    #종목 코드만 리스트화 하여 리턴
    stock_code_list = combi['stock_code'].tolist()
    
    return stock_code_list
        
    

## 가치투자 4대장 콤보

In [17]:
def stock_select(df_factor, MKTCAP_top, factor_list):
    n=50
    
    basic_list = ['stock_code', 'period', '시가총액']
    #factor_list = ['PER', 'PBR']

    basic_list.extend(factor_list)

    df_select = df_factor.copy()
    df_select = df_select[basic_list]

    df_select['score'] = 0

    # 시가총액 상위 MKTCAP_top% 산출
    #MKTCAP_top = 0.2
    df_select = df_select.sort_values(by=['시가총액'], ascending=False).head(int(len(df_select) * MKTCAP_top))
    df_select = df_select.dropna()

    # 팩터간의 점수 계산
    for i in range(len(factor_list)):
        df_select[factor_list[i] + '_score'] = (df_select[factor_list[i]] - max(df_select[factor_list[i]]))
        df_select[factor_list[i] + '_score'] = df_select[factor_list[i] + '_score']/min(df_select[factor_list[i] + '_score'])

        df_select['score'] += (df_select[factor_list[i] + '_score'] / len(factor_list))

    # 상위 n개 종목 추출
    #n = 30
    df_select = df_select.sort_values(by=['score'], ascending=False).head(n)

    # 종목 선택
    stock_select = list(df_select['stock_code'])
    
    return stock_select

## EV/EBITDA

In [18]:
def get_EVEBITDA(year):
    otp = '16555288221001111101'
    
    #시가총액 순으로 100개의 기업만 가져온다.
    #EV/EBITDA = (시가총액 + 차입금) / (영업이익 + 감가상각비)
    
    if year == '2019':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2019/12'
    elif year == '2020':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2020/12'
    elif year == '2021':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2021/12'
    else : #2022
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2022/03'
        
        
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    df_factor = pd.DataFrame()
    
    for stock_code, total in cursor :
        df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period,
                    '시가총액' : float(total)
                } , ignore_index=True)
        
    
    

## NCAV

In [19]:
def get_NCAV(year):
    
    #시가총액 순으로 100개의 기업만 가져온다.
    #NCAV = 유동자산 - 총 부채
    
    if year == '2019':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2019/12'
    elif year == '2020':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2020/12'
    elif year == '2021':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2021/12'
    else : #2022
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2022/03'
        
        
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    df_factor = pd.DataFrame()
    
    for stock_code, total in cursor :
        df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period,
                } , ignore_index=True)
    
        
    for index, row in df_factor.iterrows():
        sql='SELECT 유동자산 , 부채 FROM '+row['stock_code']+"_statement_of_position WHERE 기간 = '"+period+"';"
        
        #수집되지않은 종목 제외
        try:
            cursor.execute(sql) 
            for assets,debt  in cursor:
                    NCAV = assets - debt
                    df_factor.loc[index,'NCAV'] = NCAV

        except:
            pass
    
    #NCAV기 높을수록 안전하다고 판단
    df_factor = df_factor.sort_values(by=['NCAV'],ascending=False)
    
    top_50 = df_factor[:50]        
    
    #DB접속 해제
    cursor.close()
    conn.close()        
    
    return top_50

## PEG

In [20]:
# PEG = PER / EPS Growth 이다
#EPS Growth를 구하려면 기준 EPS 보다 4분기 전의 데이터가 필요하다
# 하지만 FnGuide에서 트레일링 데이터 자체가 1분기박에 구할 수 업기때문에
# EPS Growth를 구할 수 없으므로 PEG 또한 구할 수 없다.

## ROA

In [21]:
def get_ROA(year):
    # ROA = 자산수익률
    # ROA = 당기순이익 / 자산총액
    # 자산을 얼마나 효율적으로 운용했는지 나타내는 지표
    
    #시가총액 순으로 100개의 기업만 가져온다.
    
    if year == '2019':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2019/12'
    elif year == '2020':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2020/12'
    elif year == '2021':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2021/12'
    else : #2022
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2022/03'
        
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    df_factor = pd.DataFrame()
    
    for stock_code, total in cursor :
        df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period
                } , ignore_index=True)
        
    
    for index, row in df_factor.iterrows():
        sql='SELECT 당기순이익 FROM '+row['stock_code']+"_income_statement WHERE 기간 = '"+period+"';"
        
        #수집되지않은 종목 제외
        try:
            cursor.execute(sql) 
            for income  in cursor:
                    df_factor.loc[index,'당기순이익'] = income

        except:
            pass
    
    for index, row in df_factor.iterrows():
        sql='SELECT 자산 FROM '+row['stock_code']+"_statement_of_position WHERE 기간 = '"+period+"';"
        
        #수집되지않은 종목 제외
        try:
            cursor.execute(sql) 
            for asset  in cursor:                
                    df_factor.loc[index,'자산'] = asset
                    df_factor.loc[index,'ROA'] = row['당기순이익'] / asset[0]

                
        except:
            pass
    
    
    #ROA가 높을수록 자산을 효율적으로 운용했다고 판단
    df_factor = df_factor.sort_values(by=['ROA'],ascending=False)
    
    top_50 = df_factor[:50]    
    
    #DB접속 해제
    cursor.close()
    conn.close()        
    
    return top_50
    

## RIM

In [22]:
def get_RIM(year,earning_rate):
    
    #RIM : 잔여이익모델
    #주주가 원하는 요구 수익률보다 ROE가 높으면 그 차이를 수치화하여 기업의 가치를 평가해주는 모델
    #RIM = 장부가치 + 초과이익가치 = 자본 + (자본*((ROE-요구수익률)/요구수익률))
    #ROE = (당기순이익/자본) * 100
    #적정주가를 알고싶으면 return된 RIM에 사장주식수를 나누어주면 된다.
    
    #시가총액 순으로 100개의 기업만 가져온다.
    
    if year == '2019':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2019/12'
    elif year == '2020':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2020/12'
    elif year == '2021':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2021/12'
    else : #2022
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2022/03'
        
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    df_factor = pd.DataFrame()
    
    for stock_code, total in cursor :
        df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period
                } , ignore_index=True)
        
    for index, row in df_factor.iterrows():
        sql='SELECT 당기순이익 FROM '+row['stock_code']+"_income_statement WHERE 기간 = '"+period+"';"
        
        #수집되지않은 종목 제외
        try:
            cursor.execute(sql) 
            for income  in cursor:
                    df_factor.loc[index,'당기순이익'] = income

        except:
            pass
    
    
    for index, row in df_factor.iterrows():
        sql='SELECT 자본 FROM '+row['stock_code']+"_statement_of_position WHERE 기간 = '"+period+"';"
        
        #수집되지않은 종목 제외
        try:
            cursor.execute(sql) 
            for capital  in cursor:                
                    df_factor.loc[index,'자본'] = capital
                    df_factor.loc[index,'ROE'] = (row['당기순이익'] / capital[0]) * 100

                
        except:
            pass
    
    #RIM = 장부가치 + 초과이익가치 = 자본 + (자본*((ROE-요구수익률)/요구수익률))
    for index, row in df_factor.iterrows():
        df_factor.loc[index,'RIM'] = row['자본'] + (row['자본']*((row['ROE'] - earning_rate)/earning_rate))

    #DB접속 해제
    cursor.close()
    conn.close()     
    
    return df_factor
    

## GP/A

In [23]:
def get_GPA(year) :
    #GP/A : 매출 총 이익 / 총 자산 = (매출액 - 매출원가) / (자본 + 부채)
    
    #시가총액 순으로 100개의 기업만 가져온다.
    
    if year == '2019':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2019/12'
    elif year == '2020':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2020/12'
    elif year == '2021':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2021/12'
    else : #2022
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2022/03'
        
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    df_factor = pd.DataFrame()
    
    for stock_code, total in cursor :
        df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period
                } , ignore_index=True)
        
    for index, row in df_factor.iterrows():
        sql='SELECT 매출액, 매출원가 FROM '+row['stock_code']+"_income_statement WHERE 기간 = '"+period+"';"
        
        #수집되지않은 종목 제외
        try:
            cursor.execute(sql) 
            for sales,cost in cursor:
                    df_factor.loc[index,'매출액'] = sales
                    df_factor.loc[index,'매출원가'] = cost
                    df_factor.loc[index,'매출액-매출원가'] = sales - cost
        except:
            pass
    
    for index, row in df_factor.iterrows():
        sql='SELECT 자본, 부채 FROM '+row['stock_code']+"_statement_of_position WHERE 기간 = '"+period+"';"
        
        #수집되지않은 종목 제외
        try:
            cursor.execute(sql) 
            for assets,debt in cursor:
                    df_factor.loc[index,'자본'] = assets
                    df_factor.loc[index,'부채'] = debt
                    df_factor.loc[index,'자본-부채'] = assets - debt
        except:
            pass
    
    for index, row in df_factor.iterrows():
        df_factor.loc[index, 'GP/A'] = row['매출액-매출원가'] / row['자본-부채']
        
    
    #GP/A가 높을수록 자산대비 수익성이 좋다고 판단
    df_factor = df_factor.sort_values(by=['GP/A'],ascending=False)
    
    top_50 = df_factor[:50]    
    
    #DB접속 해제
    cursor.close()
    conn.close()        
    
    return top_50
    
    

## 부채비율

In [24]:
def get_debt_ratio(year) :
    
    #시가총액 순으로 100개의 기업만 가져온다.
    
    if year == '2019':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20191202 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2019/12'
    elif year == '2020':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20201201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2020/12'
    elif year == '2021':
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20211201 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2021/12'
    else : #2022
        sql = "SELECT stock_code, 시가총액 FROM stock_data_20220302 ORDER BY 시가총액 DESC LIMIT 100;"
        period='2022/03'
        
    conn = pymysql.connect(host = 'localhost', db = 'termproject',
                                user = 'root', passwd = '1541' , charset='utf8' , autocommit = True)
    cursor = conn.cursor()

    cursor.execute(sql)

    df_factor = pd.DataFrame()
    
    for stock_code, total in cursor :
        df_factor = df_factor.append({
                    'stock_code' : stock_code,
                    '기간' : period
                } , ignore_index=True)
        
    for index, row in df_factor.iterrows():
        sql='SELECT 부채, 자본 FROM '+row['stock_code']+"_statement_of_position WHERE 기간 = '"+period+"';"
        
        #수집되지않은 종목 제외
        try:
            cursor.execute(sql) 
            for debt,assets in cursor:
                    df_factor.loc[index,'부채'] = debt
                    df_factor.loc[index,'자본'] = assets
                    df_factor.loc[index,'부채비율'] = (debt / assets) * 100
        except:
            pass
        
    #부채비율이 낮을수록 탄탄한 기업이라고 판단
    df_factor = df_factor.sort_values(by=['부채비율'],ascending=True)
    
    top_50 = df_factor[:50]    
    
    #DB접속 해제
    cursor.close()
    conn.close()        
    
    return top_50
    

## 차입금 비율

In [25]:
# 차입금 비율 = 차입금(이자를 내는 부채) / 자본총계X100
# 차입금을 크롤링하지 못하여 차입금 비율을 계산하지 못하였습니다.

## 성장률 지표

In [26]:
# 성장률을 비교하려면 4분기 전의 데이터가 필요하지만 FnGuide 에서의 분기데이터로는 1분기 밖에 구할 수 없어
# 성장률 지표를 계산하지 못하였습니다.

## 회전률 지표

In [27]:
# 거래량 회전율 = 기간별 거래량 % 총 상장된 주식 수 * 100 
# 기간별 거래량을 크롤링 할 수 없으므로 회전률을 구할 수 없다.

## 이익률 지표

In [28]:
# 영업이익률 = 영업이익 / 매출액 * 100 = (매출액 - 매출원가 - 영업비용) / 매출액 * 100
# 매출액, 매출원가는 크롤링하여 DB에 저장하였으나 영업비용을 크롤링하지 못하여 이익률을 구할 수 없다.

In [29]:
df_PER = get_PER('2019')

In [30]:
df_PER

,stock_code,기간,당기순이익,EPS,PER
27,086790,2019/12,-32.76,-10.911198,-3299.362637
12,105560,2019/12,-278.69,-66.654389,-689.376956
31,316140,2019/12,-137.92,-19.095405,-612.712848
82,042660,2019/12,-464.85,-433.604462,-59.616545
47,028300,2019/12,-596.60,-1395.918004,-36.813051
75,009830,2019/12,-2488.65,-1541.160151,-11.478366
91,002380,2019/12,-2218.77,-21018.045754,-9.356721
58,004990,2019/12,-4116.45,-3923.829223,-9.161969
14,015760,2019/12,-22635.30,-3525.945380,-8.012036
21,035720,2019/12,-3419.25,-3972.297026,-7.907767


In [31]:
df_PBR = get_PBR('2019')

In [32]:
df_PBR

,stock_code,기간,자본,BPS,PBR
44,267250,2019/12,118847.00,729722.594034,0.094556
11,017670,2019/12,228169.00,282577.276561,0.147761
85,001040,2019/12,146151.00,500911.677006,0.187259
38,024110,2019/12,289781.00,50208.348855,0.236017
53,004020,2019/12,172337.00,129143.623638,0.246625
14,015760,2019/12,688896.00,107310.690319,0.263254
84,000100,2019/12,16499.90,129136.502023,0.293287
59,023530,2019/12,117910.00,416808.065383,0.319092
3,005380,2019/12,763658.00,357404.009959,0.339951
15,034730,2019/12,521760.00,741554.541410,0.345221


In [33]:
df_PCR = get_PCR('2019')

In [34]:
df_PCR

,stock_code,기간,시가총액,PCR
48,028300,2019/12,4.949170e+12,-101.715478
37,091990,2019/12,7.074060e+12,-43.176899
52,010140,2019/12,4.334400e+12,-6.362805
31,009540,2019/12,8.598930e+12,-5.873105
96,138930,2019/12,2.327180e+12,-4.762643
59,071050,2019/12,4.012270e+12,-3.040359
49,006800,2019/12,4.792540e+12,-1.419898
73,016360,2019/12,3.232660e+12,-0.818810
39,024110,2019/12,6.839310e+12,-0.602497
89,008560,2019/12,2.632830e+12,-0.474925


In [35]:
df_PSR = get_PSR('2019')

In [36]:
df_PSR

,stock_code,기간,시가총액,PSR
88,001040,2019/12,2.736800e+12,0.081019
98,047050,2019/12,2.270100e+12,0.092951
70,036460,2019/12,3.457120e+12,0.138381
91,001450,2019/12,2.498730e+12,0.148329
63,097950,2019/12,3.710860e+12,0.166015
16,034730,2019/12,1.801220e+13,0.184152
25,066570,2019/12,1.150440e+13,0.184643
67,139480,2019/12,3.568100e+12,0.187175
95,003490,2019/12,2.347400e+12,0.189546
54,004020,2019/12,4.250250e+12,0.207202


In [37]:
PER_and_PBR = mix_index('per','pbr','2019')

In [38]:
PER_and_PBR

['042660',
 '009830',
 '002380',
 '004990',
 '015760',
 '035720',
 '023530',
 '010140',
 '016360',
 '003490',
 '034220',
 '263750',
 '024110',
 '017670',
 '000210',
 '004170',
 '006360',
 '078930',
 '000720',
 '005380',
 '001040',
 '000270',
 '161390',
 '267250',
 '011170',
 '005490',
 '012330',
 '030200',
 '003550',
 '034730',
 '047050',
 '241560',
 '000100',
 '032640',
 '028670',
 '139480',
 '028260']

In [39]:
PCR_and_PSR = mix_index('psr','pcr','2019')

In [40]:
PCR_and_PSR

['001040',
 '047050',
 '036460',
 '001450',
 '097950',
 '034730',
 '066570',
 '139480',
 '003490',
 '267250',
 '023530',
 '005830',
 '034220',
 '006360',
 '078930',
 '030200',
 '000270',
 '015760',
 '009830',
 '005490',
 '007070',
 '042660',
 '000120',
 '011070',
 '004990',
 '004170',
 '032830',
 '032640',
 '282330',
 '009540',
 '028050',
 '010140',
 '161390',
 '035760']

In [41]:
df_NCAV = get_NCAV('2019')

In [42]:
df_NCAV

,stock_code,기간,NCAV
0,005930,2019/12,917009.00
49,006800,2019/12,119063.00
73,016360,2019/12,77953.30
6,012330,2019/12,73082.00
69,005940,2019/12,59606.20
23,033780,2019/12,44043.70
19,018260,2019/12,40877.70
50,000720,2019/12,36600.10
34,010130,2019/12,35957.70
10,005490,2019/12,35784.00


In [43]:
df_ROA = get_ROA('2020')

In [44]:
df_ROA

,stock_code,기간,당기순이익,자산,ROA
53,096530,2020/12,5031.47,10945.80,0.459671
18,036570,2020/12,5866.13,40812.10,0.143735
52,021240,2020/12,4047.11,30290.20,0.133611
56,285130,2020/12,2551.61,21199.70,0.120361
11,051900,2020/12,8131.01,68014.20,0.119549
68,011780,2020/12,5829.82,50272.80,0.115964
6,068270,2020/12,5257.03,50201.20,0.104719
58,271560,2020/12,2745.62,26654.10,0.103009
29,033780,2020/12,11716.40,114684.00,0.102162
69,000100,2020/12,1904.02,23935.20,0.079549


In [45]:
df_RIM = get_RIM('2020',10)

In [46]:
df_RIM

,stock_code,기간,당기순이익,자본,ROE,RIM
0,005930,2020/12,264078.00,2759480.00,9.569846,2640780.0
1,000660,2020/12,47589.10,519091.00,9.167776,475891.0
2,051910,2020/12,6824.08,187904.00,3.631684,68240.8
3,207940,2020/12,2409.75,45987.80,5.239977,24097.5
4,005935,2020/12,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
95,042660,2020/12,865.73,38689.70,2.237624,8657.3
96,032500,2020/12,266.38,2459.89,10.828940,2663.8
97,035760,2020/12,656.30,37851.80,1.733867,6563.0
98,023530,2020/12,-6865.70,110888.00,-6.191563,-68657.0


In [47]:
df_GPA = get_GPA('2019')

In [48]:
df_GPA

,stock_code,기간,매출액,매출원가,매출액-매출원가,자본,부채,자본-부채,GP/A
12,017670,2019/12,177407.00,0.00,177407.00,228169.00,223854.00,4315.00,41.114021
54,004020,2019/12,205126.00,191152.00,13974.00,172337.00,171326.00,1011.00,13.821958
76,000210,2019/12,16604.30,14006.20,2598.10,67367.10,67085.20,281.90,9.216389
8,051910,2019/12,273531.00,226674.00,46857.00,173838.00,166406.00,7432.00,6.304763
3,035420,2019/12,43562.40,0.00,43562.40,65039.30,57956.00,7083.30,6.150015
17,000270,2019/12,581460.00,487666.00,93794.00,289781.00,263667.00,26114.00,3.591713
11,051900,2019/12,76854.20,29172.50,47681.70,42370.90,22565.80,19805.10,2.407547
90,008930,2019/12,8166.19,0.00,8166.19,6456.87,2651.45,3805.42,2.145937
92,263750,2019/12,5359.37,0.00,5359.37,5890.17,2783.74,3106.43,1.725251
71,035760,2019/12,37897.20,23276.40,14620.80,31518.90,22830.10,8688.80,1.682718


In [49]:
df_debt = get_debt_ratio('2020')

In [50]:
df_debt

,stock_code,기간,부채,자본,부채비율
99,039490,2020/12,0.00,28567.30,0.000000
90,005940,2020/12,0.00,57977.20,0.000000
81,016360,2020/12,0.00,53170.70,0.000000
45,006800,2020/12,0.00,92688.70,0.000000
75,029780,2020/12,3.79,71195.50,0.005323
86,026960,2020/12,1285.14,13972.60,9.197572
85,095700,2020/12,603.75,5365.33,11.252803
54,035250,2020/12,3946.04,32470.30,12.152767
27,003550,2020/12,34209.80,208613.00,16.398690
40,010130,2020/12,14126.20,70870.30,19.932468
